<a href="https://colab.research.google.com/github/hiddenntreasure/Har/blob/master/HAR_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
s=pd.Series([1,2,3,4,5])
s=s.as_matrix()
t=[]
o=pd.Series([6,7,8,9,10])
o=o.as_matrix()
t.append(s)
print(t)
t.append(o)
print(t)
t

[array([1, 2, 3, 4, 5])]
[array([1, 2, 3, 4, 5]), array([ 6,  7,  8,  9, 10])]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


[array([1, 2, 3, 4, 5]), array([ 6,  7,  8,  9, 10])]

In [0]:
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

#DATA

In [0]:
DATADIR = '/content/drive/My Drive/Colab Notebooks/CS 10: Human Acitivity Recognition/UCI_HAR_Dataset'

In [0]:
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [45]:
for signal in SIGNALS:
  print(signal)

body_acc_x
body_acc_y
body_acc_z
body_gyro_x
body_gyro_y
body_gyro_z
total_acc_x
total_acc_y
total_acc_z


In [0]:
def _read_csv(filename):
  a=pd.read_csv(filename,delim_whitespace=True,header=None)
  print(a.shape)
  return a

def load_signals(subset):
  signals_data=[]
  for signal in SIGNALS:
    print(signal)
    filename = f'{DATADIR}/{subset}/Inertial Signals/{signal}_{subset}.txt'
    #print(signal)
    signals_data.append(
        _read_csv(filename).as_matrix()

    )

  return np.transpose(signals_data,(1,2,0))

In [0]:
def load_y(subset):
  filename = f'{DATADIR}/{subset}/y_{subset}.txt'
  y= _read_csv(filename)[0]
  y
  return pd.get_dummies(y).as_matrix()

In [0]:
def load_data():
  X_train,X_test = load_signals('train'), load_signals('test')
  y_train,y_test = load_y('train'),load_y('test')
  return X_train,X_test,y_train,y_test

In [8]:
np.random.seed(42) # to avoid thread
import tensorflow as tf
tf.set_random_seed(42)

In [0]:
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads = 1,
    inter_op_parallelism_threads =1
)

In [0]:
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(),config=session_conf)
K.set_session(sess)

In [0]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense,Dropout

In [0]:
epochs = 30
batch_size = 16
n_hidden = 32

In [0]:
# Count the number of classes
def _count_classes(y):
  return len(set(tuple(category) for category in y))

In [0]:
X_train,X_test,y_train,y_test = load_data()

In [49]:
X_train.shape

(7352, 128, 9)

In [0]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(y_train)

## LSTM Model

In [53]:
model = Sequential()
model.add(LSTM(n_hidden,input_shape=(timesteps,input_dim)))
model.add(Dropout(0.5))
model.add(Dense(n_classes,activation='sigmoid'))
model.summary()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [54]:
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['accuracy'])

In [55]:
model.fit(X_train,y_train,batch_size=batch_size,epochs=epochs,validation_data=(X_test,y_test))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 7352 samples, validate on 2947 samples
Epoch 1/30


7352/7352 [==============================] - 31s 4ms/step - loss: 1.3174 - acc: 0.4403 - val_loss: 1.1560 - val_acc: 0.4690
Epoch 2/30
7352/7352 [==============================] - 30s 4ms/step - loss: 1.0141 - acc: 0.5647 - val_loss: 0.9243 - val_acc: 0.6176
Epoch 3/30
7352/7352 [==============================] - 30s 4ms/step - loss: 0.8116 - acc: 0.6446 - val_loss: 0.8714 - val_acc: 0.6013
Epoch 4/30
7352/7352 [==============================] - 30s 4ms/step - loss: 0.7084 - acc: 0.6567 - val_loss: 0.7317 - val_acc: 0.6162
Epoch 5/30
7352/7352 [==============================] - 30s 4ms/step - loss: 0.6670 - acc: 0.6706 - val_loss: 0.7732 - val_acc: 0.5887
Epoch 6/30
7352/7352 [==============================] - 30s 4ms/step - loss: 0.6135 - acc: 0.6861 - val_loss: 0.6787 - val_acc: 0.6478
Epoch 7/30
7352/7352 [========================